In [4]:
import os
import pandas as pd
import sys
import sqlite3       
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")       # optional aesthetics
%matplotlib inline 

# READING THE DATA

In [5]:
data_path = r"C:\Users\dimet\Documents\GitHub\NOVAIMS_BDMwDS_PROJECT\00 Data"

In [6]:
# Function to load each Excel file into its own DataFrame
def load_excel_dataframe(filename: str, sheet_name: str) -> pd.DataFrame:
    """
    Load a sheet from an Excel file into a pandas DataFrame.
    Prints the shape on success or an error message on failure.
    """
    file_path = os.path.join(data_path, filename)
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        print(f"Loaded '{sheet_name}' from '{filename}' (shape: {df.shape})")
        return df
    except Exception as e:
        print(f"Error loading '{sheet_name}' from '{filename}': {e}")
        return pd.DataFrame()

In [7]:
# Load DataFrames
funnel_df = load_excel_dataframe("Funnel.xlsx", "funnel_data")
policies_df = load_excel_dataframe("Policy.xlsx", "policies_data")
regional_df = load_excel_dataframe("Regional.xlsx", "regional_data")

Loaded 'funnel_data' from 'Funnel.xlsx' (shape: (9373, 18))
Loaded 'policies_data' from 'Policy.xlsx' (shape: (13365, 26))
Loaded 'regional_data' from 'Regional.xlsx' (shape: (15539, 34))


# 1. DATA MERGE

In [8]:
import pandas as pd

def build_master_df(funnel_df: pd.DataFrame,
                    policies_df: pd.DataFrame,
                    regional_df: pd.DataFrame,
                    *,
                    policy_key: str = "policy_number",
                    region_key: str = "zipcode_link",
                    how: str = "left") -> pd.DataFrame:
    """
    Sequentially append policies_df and regional_df to funnel_df.

    Parameters
    ----------
    funnel_df   : pd.DataFrame  –  Base table (left‑hand frame).
    policies_df : pd.DataFrame  –  Will be joined first on `policy_number`.
    regional_df : pd.DataFrame  –  Will be joined second on `zipcode_link`.
    policy_key  : str           –  Key column in both funnel_df & policies_df.
    region_key  : str           –  Key column in both intermediate & regional_df.
    how         : {"left","inner","right","outer"}  –  Join strategy.
                                                  
    Returns
    -------
    pd.DataFrame – Enriched dataframe containing all columns from the three inputs.
    """

    # --- 1. Join policies data ---------------------------------------------
    merged = (
        funnel_df
        .merge(
            policies_df.drop_duplicates(subset=policy_key),
            on=policy_key,
            how=how,
            suffixes=("", "_pol")      # tag colliding names from policies_df
        )
    )

    # --- 2. Join regional data ---------------------------------------------
    merged = (
        merged
        .merge(
            regional_df.drop_duplicates(subset=region_key),
            on=region_key,
            how=how,
            suffixes=("", "_reg")      # tag colliding names from regional_df
        )
    )

    return merged

In [9]:
master_df = build_master_df(funnel_df, policies_df, regional_df)

In [10]:
master_df

,affinity_name,status_report,offer_number,policy_number,zipcode_link,zip4,birth_date,brand,date_offer,date_request,...,FAM_CHILD_Y,FAM_CHILD_O,FAM_WCHILD_Y,FAM_WCHILD_MED,FAM_WCHILD_OLD,CIT_HOUSEHOLD,LOAN,SAVINGS,SHOP_ONLINE,CAR
0,Insuro,Requestwithdrawn,1000,10000.0,10000,2132,1985-01-01,HYUNDAI,2018-10-11,2018-10-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,other,Tailoredofferwithdrawn,1001,NaN,10001,6027,1987-04-01,AUDI,2018-10-11,NaT,...,4.0,3.0,1.0,1.0,4.0,5.0,2.0,6.0,5.0,5.0
2,other,Incompleterequest,1002,NaN,10002,3824,1972-11-01,VOLKSWAGEN,2018-10-11,NaT,...,4.0,4.0,1.0,2.0,2.0,6.0,3.0,5.0,6.0,5.0
3,other,Policycreated,1003,10002.0,10003,6921,1983-08-01,MAZDA,2018-10-11,2018-10-11,...,4.0,3.0,2.0,3.0,3.0,5.0,3.0,5.0,6.0,5.0
4,other,Policycreated,1004,10003.0,10004,8266,1990-04-01,VOLVO,2018-10-12,2018-10-12,...,3.0,3.0,2.0,3.0,3.0,5.0,4.0,4.0,6.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9368,other,Requestaccepted,9439,11946.0,15568,1161,1960-09-01,VOLKSWAGEN,2020-03-08,2020-03-08,...,2.0,4.0,1.0,2.0,3.0,6.0,4.0,5.0,6.0,5.0
9369,other,Waitforapproval,9440,11947.0,15582,5015,1953-04-01,NISSAN,2020-03-08,2020-03-08,...,3.0,3.0,1.0,3.0,4.0,5.0,4.0,6.0,6.0,5.0
9370,Insuro,Tailoredofferrequested,9441,NaN,10332,3078,1976-04-01,TOYOTA,2020-03-08,NaT,...,4.0,3.0,2.0,3.0,3.0,5.0,5.0,4.0,6.0,4.0
9371,T&B,Calculatenewpremium,9442,NaN,12968,1965,1951-07-01,TOYOTA,2020-03-08,NaT,...,1.0,3.0,1.0,3.0,4.0,3.0,2.0,3.0,3.0,5.0


In [11]:
master_df["conv"] = np.where(
    master_df["status_report"].eq("Policycreated"),  # condition
    1,                                               # value if True
    0                                                # value if False
)

In [12]:
master_df['policy_start_date'] = pd.to_datetime(master_df['policy_start_date'], errors='coerce')
master_df['policy_start_date_pol'] = pd.to_datetime(master_df['policy_start_date_pol'], errors='coerce')

In [13]:
# 2) Define the cutoff date:
threshold = pd.to_datetime('2019-11-01')

# 3) Apply np.where:
master_df['churn'] = np.where(
    # First np.where‐condition: missing OR on/after 2019-11-01 → churn = -1
    (master_df['policy_start_date'].isna()) 
    | 
    (master_df['policy_start_date'] >= threshold),
    -1,

    # Else, check second condition: missing policy_start_date_pol → churn = 1; otherwise 0
    np.where(
        master_df['policy_start_date_pol'].isna(),
        1,
        0
    )
)

In [14]:
master_df[['policy_start_date','policy_start_date_pol',"churn"]].head()

,policy_start_date,policy_start_date_pol,churn
0,2018-11-01,NaT,1
1,NaT,NaT,-1
2,NaT,NaT,-1
3,2018-10-12,NaT,1
4,2018-11-01,NaT,1


In [21]:
csv_name    = "Dataset.csv"                           # final file name
csv_file    = os.path.join(data_path, csv_name)       # full destination path

# ── Persist ───────────────────────────────────────────────────────────────
# Write the DataFrame to CSV (no index column by default)
master_df.to_csv(csv_file, index=False)

print(f"Saved {len(master_df):,} rows to {csv_file}")

Saved 9,373 rows to C:\Users\dimet\OneDrive - NOVAIMS\NOVA IMS BDMWDS\0 Data\Dataset.csv
